In [ ]:
import requests
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import read_excel
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
chromedriver = "C:/Users/Kernius/Desktop/BoredPanda/chromedriver"

In [ ]:
A = pd.read_excel("C:\\Users\\Kernius\\Desktop\\Bored\\Table_A.xlsx", Header = 1)
B = pd.read_excel("C:\\Users\\Kernius\\Desktop\\Bored\\Table_B.xlsx", Header = 1)

In [ ]:
AB = A.merge(B, how = 'inner', left_on = 'video_id', right_on = 'video_id')

In [ ]:
print(len(A) - len(AB), "rows in A, but not in B table")
print(len(AB), "rows shared by A and B tables")

In [ ]:
data = pd.concat([A, B]).drop_duplicates()
data['published_at'] = pd.to_datetime(data['published_at']).dt.date

In [ ]:
print(len(data), "rows in joined A and B tables without duplicates")

In [ ]:
print("In", data['video_id'].groupby(data['published_at']).count().idxmax(),"was published" , data['video_id'].groupby(data['published_at']).count().max(), "videos")

In [ ]:
data["Size"] = np.where(data['views']<1000000, 'small', 'big')

In [ ]:
A = pd.read_excel("C:\\Users\\Kernius\\Desktop\\Bored\\Table_A.xlsx", Header = 1).dropna()
B = pd.read_excel("C:\\Users\\Kernius\\Desktop\\Bored\\Table_B.xlsx", Header = 1).dropna()
data = pd.concat([A, B]).drop_duplicates()
data.reset_index(drop=True, inplace=True)  ## Reikėjo resetint indexą, nes kažkoks bugas buvo, kad peršoko 12 indexo reikšmę (ilgai užtrukau kol atradau šitą nesąmonę)
data["Size"] = np.where(data['views']<1000000, 'small', 'big')

In [ ]:
link = []
for i in range(len(data)):
    
    link.append("https://www.youtube.com/watch?v=" + data['video_id'][i])
## pasidarom link'ų list'ą

In [ ]:
new_views = pd.DataFrame(columns = ['video_id', 'n_views'])
driver = webdriver.Chrome(chromedriver) ## būtinai turi būt įjungta, kad veiktų tolimesnis kodas

In [ ]:
wait = WebDriverWait(driver, 10)

for x in link:
            r = requests.get(x)
            if "Video unavailable" in r.text or "nepasiekiamas" in r.text or "private video" in r.text or "if you've been granted access to this video" in r.text or "privatus" in r.text:
                continue
            else:
                driver.get(x)
                id = x.strip('https://www.youtube.com/watch?v=')
                n_views =  wait.until(EC.presence_of_element_located(
                                             (By.CSS_SELECTOR,"div#count yt-view-count-renderer"))).text
                new_views.loc[len(new_views)] = [id, n_views]

In [ ]:
new_views['n_views'] = new_views['n_views'].str.rstrip('views')
for i in range(len(new_views)):
    new_views['n_views'][i] = int(new_views['n_views'].str.rstrip('views')[i].replace(',' , ''))
    
## konvertuojam peržiūras į integer

In [ ]:
print("Video with max views is this: ",data['video_id'][new_views['n_views'].astype(str).astype(int).idxmax()] , "with", new_views['n_views'].max(), "views")

In [ ]:
print("Previous video with max views was this:", data['video_id'][data['views'].idxmax()], "with", data['views'].max(), "views")


In [ ]:
new_data = data.merge(new_views, how = 'inner', left_on = 'video_id', right_on = 'video_id')


In [ ]:
new_data.to_csv("TableAB_bonus.csv")

In [ ]:
C = pd.read_excel("C:\\Users\\Kernius\\Desktop\\Bored\\TableAB_bonus.xlsx", Header = 1)


In [ ]:
bins = ([0, 500000, 1000000, 2000000, 3000000,4000000])
plt.hist([C['views'], C['n_views']], bins,label=['Old views', 'New views'])

plt.title('Old and new views histogram')
plt.legend(loc='upper right')
plt.ticklabel_format(style='plain')
plt.xticks(rotation='vertical')

In [ ]:
print("Didžioji dalis video turi peržiūrų iki 500 000")